<a href="https://colab.research.google.com/github/YYx00xZZ/colabBackup/blob/master/pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://machinelearningmastery.com/how-to-predict-whether-eyes-are-open-or-closed-using-brain-waves/

# https://academic.oup.com/gigascience/article/6/7/gix034/3796323

### http://gigadb.org/dataset/view/id/100295/Sample_page/5

    'time_size' : 128, # row count for 1 sec
    'offset' : 0.75, # Отместване в секунди на началото на всяка епоха; ! Cqlo chislo ili gurmi
    'epoch' : 3, # Дължина на епохата по време на експеримента в сек
    
    # Да се добавят стойности
    'chunk_duration_sec_IN' : None, # Дължина на епохата по време на експеримента
    'epoch_duration_IN' : None, # дължина на епохата в редове

In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from plotly import tools
from scipy import signal
from plotly.graph_objects import Layout, YAxis, Scatter, Annotation, Annotations, Data, Figure, Marker, Font
%matplotlib inline

In [ ]:
# plt.rcParams['figure.figsize'] = [16, 9]

# DATA CLEANING


In [ ]:
# ## Друг начин за генериране на time vector: 
# fs = 128
# times = create_times(len(sig)/fs, fs)


### Промяна в начина по който третираме празните полета в колона `EventId`

## важно (unique() ids)

### Нужна е промяна в метода на обработка на колона EventId. Редове със стойност - `32780:32778:800`, `768:786` или `33282:768` биват изтрити.
В един момент от времето - няколко евента  
1. Добавяме колона и разделяме евентите в две колони ,например редове с код за ляво и дясно - `EventId`, `experiment_info` съдържаща останалите.
2. Времева стъпка, разширение


# config & predefined funcs

In [ ]:
target_url = "/content/drive/My Drive/Python_Reserch/Dimi_Data/2_KA_VisualnaStimulaciq_14ch.csv"
all_cols = ['Time128Hz', 'Epoch', 'AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'EventId', 'EventDate', 'EventDuration']
# channel_columns = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
channel_columns = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
# 32780:32778:800
 
stimulation_codes = {
        '33025': 'OVTK_StimulationId_Label_01',
        '33026': 'OVTK_StimulationId_Label_02',
        '32769': 'OVTK_StimulationId_ExperimentStart',
        '32775': 'OVTK_StimulationId_BaselineStart',
        '32776': 'OVTK_StimulationId_BaselineStop',
        '33282': 'OVTK_StimulationId_Beep',
        '768': 'OVTK_GDF_Start_Of_Trial',
        '786': 'OVTK_GDF_Cross_On_Screen',
        '32780': 'OVTK_StimulationId_VisualStimulationStop',
        '32777': 'OVTK_StimulationId_RestStart',
        '1010': 'OVTK_GDF_End_Of_Session',
        '32770': 'OVTK_StimulationId_ExperimentStop',
        '800': 'OVTK_GDF_End_Of_Trial',
        '32778': 'OVTK_StimulationId_RestStop'
    }
    # '32780:32778:800',
    # '768:786',
    # '33282:768'
id_remap = {
        '33025':' 10',
        '33026':' 20',
        '32769':' 30',
        '32775':' 40',
        '32776':' 50',
        '33282':' 60',
        '768':' 70',
        '786':' 80',
        '32780':'90',
        '32777':'110',
        '1010':'130',
        '32770':'140',
        0:' 100',
        1:'120',
    }
col_names = ['Time128Hz',
              'Epoch',
              'AF3',
              'F7',
              'F3',
              'FC5',
              'T7',
              'P7',
              'O1',
              'O2',
              'P8',
              'T8',
              'FC6',
              'F4',
              'F8',
              'AF4',
              'EventId',
              'EventDate',
              'EventDuration']

In [ ]:
 
 
#raw_data = '/content/2_KA_VisualnaStimulaciq_14ch.csv'
 
def interpolate_eventid_col(data):
  data.fillna(value=0, inplace=True)#, inplace=True)
    # FILL-EXPLODE EVENTID
  # this should be a func
  data.columns = data.columns.str.replace(' ', '')
  EventID_NEW = ''
  EventID_TO_SET = ''
  for index, row in data.iterrows(): #итерираме ред по ред с вградения метод iterrows() от pandas
      EventID_NEW = str(row['EventId']) # достъпваме данни чрез име на колона
      if EventID_NEW == '0':
          data.at[index, 'EventId'] = str(EventID_TO_SET)
      else:
          EventID_NEW = str(row['EventId'])
          EventID_TO_SET = str(row['EventId'])
  return data
 
 
def fix_events(data):
    data = data[
        (data.EventId == '33025') | \
        (data.EventId == '33026') | \
        (data.EventId == '32769') | \
        (data.EventId == '32775') | \
        (data.EventId == '32776') | \
        (data.EventId == '33282') | \
        (data.EventId == '768') | \
        (data.EventId == '786') | \
        (data.EventId == '32780') | \
        (data.EventId == 0) | \
        (data.EventId == '32777') | \
        # (data.EventId == 1) | \
        (data.EventId == '1010') | \
        (data.EventId == '32770')
    ]
    # data = data.astype({'EventId': 'int32'}).dtypes
    return data
    # pass
 
 
def eventid_to_int(data):
    for index, row in data.iterrows():
        EventID_value = str(row['EventId'])
        data.at[index, 'EventId'] = int(EventID_value)
    data = data.astype({'EventId': int})
    return data
 
 
def label_data(data):
  for index, row in data.iterrows(): #итерираме ред по ред с вградения метод iterrows() от pandas
      EventID_value = str(row['EventId']) # достъпваме данни чрез име на колона
      if EventID_value == '33025':
          # data.at[index, 'EventId'] = str(EventID_TO_SET)
          data.at[index, 'EventId'] = 0
      elif EventID_value == '33026':
          data.at[index, 'EventId'] = 1
      else:
        #   data.at[index, 'EventId'] = -1
        data.at[index, 'EventId'] = int(EventID_value)
  return data
 
 
def decode_stimula_codes(data, stimulation_codes):
    data['StimulationCode'] = data['EventId'].map(stimulation_codes)
    return data
 
 
def test_func(data):
    # frame = {} #initialize empty dict to store filtered cols
    # # m = len(data[channel_columns])
    # for index, row in data.iterrows():
    #     EventID_value = str(row['EventId'])
    #     data.at[index, 'EventId'] = int(EventID_value)
    #     print(column)
    pass
 
 
def butter_bandpass(lowcut, highcut, fs, order=6):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a
 
 
def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.lfilter(b, a, data)
    return y
 
 
def do_filter(data,lowcut,highcut,fs,order, channel_columns):
    frame = {} 
    # initialize empty dict to store filtered cols
    # channel_columns - само колоните, които съдържат данните от предварително избраните от нас канали , посочени в конфиг файла
    for column in data[channel_columns]:               
        # data[column] - избраните колони 
        # lowcut - долна граница
        # highcut - гора граница
        # fs - ???
        # order - ред за прилагане на филтъра
        # y - данни за поредната колона от списъка channel_columns
        y = butter_bandpass_filter(data[column], lowcut, highcut, fs, order=order)
        # filtered_column - образуваме серия, т.е. всяка колона след филтриране я превръщаме в серия - подреден списък, специфичен за пандас
        filtered_column = pd.Series(data=y) #, name=str(column)
        # frame.update - упдейтваме , като заместваме данните във всяка подредена филтрирана колона 
        frame.update( { str(column) : filtered_column } )
    # извлечените филтрирани колони , използвайки предварително дефинирания фрейм 
    butter_data = pd.DataFrame(frame)
    butter_data.insert(0,'Time128Hz',data['Time128Hz'])
    butter_data.append(data['EventId'])
    butter_data.append(data['EventDate'])
    butter_data.append(data['EventDuration'])
    return butter_data
 
 
def outliers_remover(data,channel_columns):
    frame = {} #initialize empty dict to store filtered cols
 
    #     channel_columns - само колоните, които съдържат данните от предварително избраните от нас канали , посочени в конфиг файла
    for column in data[channel_columns]:               
        # data[column] - избраните колони 
 
        # y - данни за поредната колона от списъка channel_columns
        y = data[column]
        data_mean, data_std = np.mean(y), np.std(y)
        cut_off = data_std * 4
        lower, upper = data_mean - cut_off, data_mean + cut_off
        # print(lower, upper)
        outliers = [x for x in y if x < lower or x > upper]
        # remove outliers
        outliers_removed = [x for x in y if x >= lower and x <= upper]
        filtered_col = pd.Series(data=outliers_removed)
        frame.update( { str(column) : filtered_col } )
        aaa = pd.DataFrame(frame)
        aaa.insert(0,'Time128Hz',data['Time128Hz'])
    return aaa
 
 
def dsp(data,channel_columns):
    """Функция - DSP x*x
 
    Извлича данните от CSV файла
 
    Todo:
    * For module TODOs
    * You have to also use ``sphinx.ext.todo`` extension
 
    .. _Google Python Style Guide:
    http://google.github.io/styleguide/pyguide.html
    """
    frame = {} #initialize empty dict to store filtered cols
 
    #     channel_columns - само колоните, които съдържат данните от предварително избраните от нас канали , посочени в конфиг файла
    for column in data[channel_columns]:               
        # data[column] - избраните колони 
 
        # y - данни за поредната колона от списъка channel_columns
        y = data[column]
        y = y*y
        #     filtered_column - образуваме серия, т.е. всяка колона след филтриране я превръщаме в серия - подреден списък, специфичен за pandas
        filtered_column = pd.Series(data=y) #, name=str(column)
        # frame.update - упдейтваме , като заместваме данните във всяка подредена филтрирана колона 
        frame.update( { str(column) : filtered_column } )
    #print(frame)
 
    #     извлечените филтрирани колони , използвайки предварително дефинирания фрейм 
    dsp_data = pd.DataFrame(frame)
    dsp_data.insert(0,'Time128Hz',data['Time128Hz'])
    return dsp_data
    #     Добавяме съответно 
    #     Time128Hz
    #     Epoch
    #     EventId
    # Return_Data.insert(0,'Time128Hz',data['Time128Hz'])
 
 
def fft_x(data,channel_columns):
 
    frame = {} #initialize empty dict to store filtered cols
 
    #     channel_columns - само колоните, които съдържат данните от предварително избраните от нас канали , посочени в конфиг файла
    for column in data[channel_columns]:               
        # data[column] - избраните колони 
 
        # y - данни за поредната колона от списъка channel_columns
        y = data[column]
       
        y = np.fft.fft(y)
        #     filtered_column - образуваме серия, т.е. всяка колона след филтриране я превръщаме в серия - подреден списък, специфичен за pandas
        filtered_column = pd.Series(data=y) #, name=str(column)
        # frame.update - упдейтваме , като заместваме данните във всяка подредена филтрирана колона 
        frame.update( { str(column) : filtered_column } )
    #print(frame)
 
    #     извлечените филтрирани колони , използвайки предварително дефинирания фрейм 
    fft_data = pd.DataFrame(frame)
    #     Добавяме съответно 
    #     Time128Hz
    #     Epoch
    #     EventId
    fft_data.insert(0,'Time128Hz',data['Time128Hz'])
    return fft_data

In [ ]:
#@title
# import numpy as np

# fs = 512                                # Sampling rate (512 Hz)
# data = np.random.uniform(0, 100, 1024)  # 2 sec of data b/w 0.0-100.0
# print(data)
# print(len(data))
# print()
# # Get real amplitudes of FFT (only in postive frequencies)
# fft_vals = np.absolute(np.fft.rfft(data))

# # Get frequencies for amplitudes in Hz
# fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
# print(fft_freq)
# print()
# # Define EEG bands
# eeg_bands = {'Delta': (0, 4),
#              'Theta': (4, 8),
#              'Alpha': (8, 12),
#              'Beta': (12, 30),
#              'Gamma': (30, 45)}

# # Take the mean of the fft amplitude for each EEG band
# eeg_band_fft = dict()
# for band in eeg_bands:  
#     freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
#                        (fft_freq <= eeg_bands[band][1]))[0]
#     eeg_band_fft[band] = np.mean(fft_vals[freq_ix])

# # Plot the data (using pandas here cause it's easy)
# import pandas as pd
# df = pd.DataFrame(columns=['band', 'val'])
# df['band'] = eeg_bands.keys()
# df['val'] = [eeg_band_fft[band] for band in eeg_bands]
# ax = df.plot.bar(x='band', y='val', legend=False)
# ax.set_xlabel("EEG band")
# ax.set_ylabel("Mean band Amplitude")

# read data

In [ ]:
# we use pandas to read the dataset and specify its feature names
df = pd.read_csv(target_url, names=col_names, index_col=False, header=0, sep=',')
# df.fillna(value=0, inplace=True)

In [ ]:
df = interpolate_eventid_col(df)
df.head()
# df.fillna(value=0, inplace=True)
# values = df.to_numpy()
# plt.figure()
# for i in range(values.shape[1]):
#     plt.subplot(values.shape[1], 1, i+1)
#     plt.plot(values[:, i])
# plt.show()
# df['ez_id'] = df['EventId'].map(id_remap)
# # df.EventId.unique()

,Time128Hz,Epoch,AF3,F7,F3,FC5,T7,P7,O1,O2,P8,T8,FC6,F4,F8,AF4,EventId,EventDate,EventDuration
0,0.000000,0,4114.102539,4110.000000,4118.974121,4104.487305,4127.820312,4112.948730,4106.794922,4123.077148,4150.641113,4158.461426,4123.717773,4119.102539,4139.230957,4117.436035,,0,0
1,0.007812,0,4113.974121,4107.692383,4121.538574,4106.538574,4114.358887,4113.974121,4108.333496,4128.333496,4155.512695,4158.717773,4128.717773,4116.410156,4137.436035,4115.769043,32769,0.0089665661,0.0000000000
2,0.015625,0,4107.820312,4105.897461,4114.615234,4105.512695,4111.410156,4107.179688,4103.077148,4121.410156,4136.922852,4148.846191,4120.128418,4101.538574,4125.384766,4108.717773,32769,0,0
3,0.023438,0,4106.666504,4105.897461,4110.000000,4104.871582,4127.436035,4107.179688,4106.410156,4115.384766,4120.512695,4136.538574,4105.512695,4101.410156,4120.512695,4107.563965,32769,0,0
4,0.031250,0,4114.102539,4109.102539,4119.358887,4109.487305,4129.358887,4118.974121,4118.974121,4117.307617,4120.641113,4144.615234,4107.307617,4113.846191,4123.077148,4112.563965,32769,0,0


In [ ]:
df = fix_events(df)
df.shape

(67433, 19)

In [ ]:
df =  eventid_to_int(df)
# df.head()
# # plt.plot(df.Time128Hz[:3560].to_numpy(), df.EventId[:3560].to_numpy())
# omg = df.ez_id
# tomg = df.Time128Hz
# # omg = [if x%2 == 0:x/100 else x/1000*1.2 for x in omg]
# plt.plot(tomg,omg)
# # print(int(0.768**2))

In [ ]:
df = do_filter(df,8,12,128,6,channel_columns)
df.head()

In [ ]:
df = dsp(df, channel_columns)
# dsp_data.describe()

In [ ]:
aaa = outliers_remover(dsp_data, channel_columns)
# aaa.describe()

In [ ]:
fft_data = fft_x(dsp_data, channel_columns)
# fft_data.head()
# sig_fft = fft_data
# power = np.abs(sig_fft)**2
# print(power)

In [ ]:
#@title
# # dt = 0.001
# dt = 0.0078125
# t = df.Time128Hz.to_numpy()
# f = df.AF3.to_numpy()
# f_clean = f
# n = len(t)
# fhat = np.fft.fft(f,n)
# PSD = fhat * np.conj(fhat) / n
# freq = (1/(dt*n)) * np.arange(n)
# L = np.arange(1,np.floor(n/2),dtype='int')

# fix,axs = plt.subplots(2,1)

# plt.sca(axs[0])
# plt.plot(t,f,color='c',LineWidth=1.5,label='Noisy')
# plt.plot(t,f_clean,color='k',LineWidth=2,label='clean')
# plt.xlim(t[0], t[-1])
# plt.legend()


# plt.sca(axs[1])
# plt.plot(freq[L],PSD[L],color='c',LineWidth=2,label='Noisy')
# plt.xlim(freq[L[0]],freq[L[-1]])
# plt.legend()
# plt.show()
# print(freq)

In [ ]:
# df.to_csv('formatlab.csv',sep=',', index=None)

# BANDPASS

In [ ]:
#@title
# from scipy.signal import butter, lfilter

# def butter_bandpass(lowcut, highcut, fs, order=5):
#     nyq = 0.5 * fs
#     low = lowcut / nyq
#     high = highcut / nyq
#     b, a = signal.butter(order, [low, high], btype='band')
#     return b, a


# def butter_bandpass_filter(data, lowcut, highcut, fs, order=6):
#     b, a = butter_bandpass(lowcut, highcut, fs, order=order)
#     y = signal.lfilter(b, a, data)
#     return y


# # # def run():
#     # import numpy as np
#     # import matplotlib.pyplot as plt
#     # from scipy.signal import freqz
#     # import math
#     # # Sample rate and desired cutoff frequencies (in Hz).
#     # fs = 128.0
#     # lowcut = 8.0
#     # highcut = 30.0

#     # # -----------------------------------------------------------------------------
#     #     # # Plot the frequency response for a few different orders.
#     #     # plt.figure(1)
#     #     # plt.clf()
#     #     # for order in [3, 6, 9]:
#     #     #     b, a = butter_bandpass(lowcut, highcut, fs, order=order)
#     #     #     w, h = freqz(b, a, worN=2000)
#     #     #     plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)

#     #     # plt.plot([0, 0.5 * fs], [np.sqrt(0.5), np.sqrt(0.5)],
#     #     #     '--', label='sqrt(0.5)')
#     #     # plt.xlabel('Frequency (Hz)')
#     #     # plt.ylabel('Gain')
#     #     # plt.grid(True)
#     #     # plt.legend(loc='best')

#     #     # # Filter a noisy signal.
#     #     # T = 1
#     #     # nsamples = int(T * fs)
#     #     # xd = 2 * math.pi / T
#     #     # print(xd)
#     #     # # t = np.linspace(0, T, nsamples, endpoint=False)
#     #     # t = df.Time128Hz[:nsamples].to_numpy()
#     #     # a = 0.02
#     #     # f0 = 16.0
#     #     #     # x = 0.1 * np.sin(2 * np.pi * 1.2 * np.sqrt(t))
#     #     #     # x += 0.01 * np.cos(2 * np.pi * 312 * t + 0.1)
#     #     #     # x += a * np.cos(2 * np.pi * f0 * t + .11)
#     #     #     # x += 0.03 * np.cos(2 * np.pi * 2000 * t)
#     #     # x = signals[:nsamples, 2]
#     #     # plt.figure(2)
#     #     # plt.clf()
#     #     # plt.plot(t, x, label='Noisy signal')

#     #     # y = butter_bandpass_filter(x, lowcut, highcut, fs, order=6)
#     #     # plt.plot(t, y, label='Filtered signal (%g Hz)' % f0)
#     #     # plt.xlabel('time (seconds)')
#     #     # plt.hlines([-a, a], 0, T, linestyles='--')
#     #     # plt.grid(True)
#     #     # plt.axis('tight')
#     #     # plt.legend(loc='upper left')

#     #     # plt.show()

#     # # -----------------------------------------------------------------------------

# def do_filter(data,lowcut,highcut,fs,order):
#     frame = {} 
#     # initialize empty dict to store filtered cols


#     # channel_columns - само колоните, които съдържат данните от предварително избраните от нас канали , посочени в конфиг файла
#     for column in df[channel_columns]:               
#         # data[column] - избраните колони 
#         # lowcut - долна граница
#         # highcut - гора граница
#         # fs - ???
#         # order - ред за прилагане на филтъра

#         # y - данни за поредната колона от списъка channel_columns
#         y = butter_bandpass_filter(df[column], lowcut, highcut, fs, order=6)
#         # filtered_column - образуваме серия, т.е. всяка колона след филтриране я превръщаме в серия - подреден списък, специфичен за пандас
#         filtered_column = pd.Series(data=y) #, name=str(column)
#         # frame.update - упдейтваме , като заместваме данните във всяка подредена филтрирана колона 
#         frame.update( { str(column) : filtered_column } )

#     #     извлечените филтрирани колони , използвайки предварително дефинирания фрейм 
#     butter_data = pd.DataFrame(frame)
#     butter_data.insert(0,'Time128Hz',data['Time128Hz'])
#     return butter_data

In [ ]:
#@title
# df['EventId'].unique()
# dataframe = df[
#     (df.EventId == '33025') | \
#     (df.EventId == '33026') | \
#     (df.EventId == '32769') | \
#     (df.EventId == '32775') | \
#     (df.EventId == '32776') | \
#     (df.EventId == '33282') | \
#     (df.EventId == '768') | \
#     (df.EventId == '786') | \
#     (df.EventId == '32780') | \
#     (df.EventId == 0) | \
#     (df.EventId == '32777') | \
#     (df.EventId == 1) | \
#     (df.EventId == '1010') | \
#     (df.EventId == '32770')
#     ]
# stimulation_codes = {
#     '33025': 'OVTK_StimulationId_Label_01',
#     '33026': 'OVTK_StimulationId_Label_02',
#     '32769': 'OVTK_StimulationId_ExperimentStart',
#     '32775': 'OVTK_StimulationId_BaselineStart',
#     '32776': 'OVTK_StimulationId_BaselineStop',
#     '33282': 'OVTK_StimulationId_Beep',
#     '768': 'OVTK_GDF_Start_Of_Trial',
#     '786': 'OVTK_GDF_Cross_On_Screen',
#     '32780': 'OVTK_StimulationId_VisualStimulationStop',
#     0: 'OVTK_StimulationId_Number_00',
#     '32777': 'OVTK_StimulationId_RestStart',
#     1: 'OVTK_StimulationId_Number_01',
#     '1010': 'OVTK_GDF_End_Of_Session',
#     '32770': 'OVTK_StimulationId_ExperimentStop'
# }
# dataframe['StimulaCode'] = dataframe['EventId'].map(stimulation_codes)
# dataframe.head()

In [ ]:
#@title
#
    # times = df.Time128Hz.to_numpy()
    # t = times
    # n = len(t)
    # f = signals[:, 2]
    # fhat = np.fft.fft(f,n)
    # PSD = fhat * np.conj(fhat) / n
    # freq = (1/(dt*n)) * np.arange(n)
    # L = np.arange(1,np.floor(n/2),dtype='int')

    # fix,axs = plt.subplots(2,1)

    # plt.sca(axs[0])
    # plt.plot(t,f,color='c',LineWidth=1.5,label='Noisy')
    # plt.plot(t,f_clean,color='k',LineWidth=0.5,label='clean')
    # plt.xlim(t[0], t[-1])
    # plt.legend()


    # plt.sca(axs[1])
    # plt.plot(freq[L],PSD[L],color='c',LineWidth=2,label='Noisy')
    # plt.xlim(freq[L[0]],freq[L[-1]])
    # plt.legend()
    # plt.show()

In [ ]:
# this line displays the first five rows of the dataset
times = df.Time128Hz.to_numpy()
signals_df = df[
                ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4'] # , 'EventId'
                ]

ch_names = ['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']
dsp_signals = dsp_data[ch_names].to_numpy()

In [ ]:
# from plotly.subplots import make_subplots
    # import re

    # signals = signals.T
    # signals = signals[:128]


    # fig = make_subplots(
    #     rows=signals.shape[1], cols=1,
    #     shared_xaxes=True,
    #     vertical_spacing=0.001
    #     )

    # for i in range(signals.shape[1]):
    #     fig.add_trace(
    #         go.Scatter(
    #             x=times,
    #             y=signals[:, i],
    #             mode="lines",
    #             name="Channel: " + ch_names[i-1]
    #         ),
    #         row=i+1, col=1
    #     )

    # # Set theme, margin, and annotation in layout
    # fig.update_layout(
    #     # height=1400,
    #     template="plotly_dark",
    #     margin=dict(r=10, t=10, b=20, l=60),
    #     annotations=[
    #         dict(
    #             text="Motor-Imagery Left/Right hand movement",
    #             showarrow=False,
    #             xref="paper",
    #             yref="paper",
    #             x=0,
    #             y=0)
    #     ]
    # )

    # # fig.update_layout(
    # #     height=1400,
    # #     showlegend=False,
    # #     title_text="Channels",
    # # )

In [ ]:
plt.figure()
for i in range(dsp_signals.shape[1]):
	plt.subplot(dsp_signals.shape[1], 1, i+1)
	plt.plot(dsp_signals[:, i])
plt.show()

## outliners removal

In [ ]:
#@title
# from numpy import mean
# from numpy import std
# from numpy import delete
# from numpy import savetxt
# # load the dataset.
# # data = read_csv('EEG_Eye_State.csv', header=None)
# values = df[['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4']].to_numpy
# # step over each EEG column
# for i in range(values.shape[1] - 1):
# 	# calculate column mean and standard deviation
# 	data_mean, data_std = mean(values[:,i]), std(values[:,i])
# 	# define outlier bounds
# 	cut_off = data_std * 4
# 	lower, upper = data_mean - cut_off, data_mean + cut_off
# 	# remove too small
# 	too_small = [j for j in range(values.shape[0]) if values[j,i] < lower]
# 	values = delete(values, too_small, 0)
# 	print('>deleted %d rows' % len(too_small))
# 	# remove too large
# 	too_large = [j for j in range(values.shape[0]) if values[j,i] > upper]
# 	values = delete(values, too_large, 0)
# 	print('>deleted %d rows' % len(too_large))
# 	# concat_outliners = too_small
# 	# concat_outliners.extend(too_large)
# 	# print(type(concat_outliners))
# 	# print(len(concat_outliners))
# # save the results to a new file
# # savetxt('EEG_Eye_State_no_outliers.csv', values, delimiter=',')

## outliners remover

In [ ]:
# def outliers_remover(data, channel_columns):
    #     frame = {} #initialize empty dict to store filtered cols

    #     #     channel_columns - само колоните, които съдържат данните от предварително избраните от нас канали , посочени в конфиг файла
    #     for column in data[channel_columns]:               
    #         # data[column] - избраните колони 

    #         # y - данни за поредната колона от списъка channel_columns
    #         y = data[column]
    #         data_mean, data_std = np.mean(y), np.std(y)
    #         cut_off = data_std * 3
    #         lower, upper = data_mean - cut_off, data_mean + cut_off
    #         # print(lower, upper)
    #         outliers = [x for x in y if x < lower or x > upper]
    #         # remove outliers
    #         outliers_removed = [x for x in y if x >= lower and x <= upper]
    #         filtered_col = pd.Series(data=outliers_removed)
    #         frame.update( { str(column) : filtered_column } )
    #         aaa = pd.DataFrame(frame)
    #         aaa.insert(0,'Time128Hz',data['Time128Hz'])
    #     return aaa

# aaa = outliers_remover(df, channel_columns)
# aaa.to_csv('data.csv', index=None, sep=',')
aaa.head()

In [ ]:
df.head()

In [ ]:
# seconds = 128 * 0.2
# plt.plot(df.Time128Hz, df[:,:-1],color='black',LineWidth=1.0,label='Noisy')
# plt.plot(aaa.Time128Hz, aaa.AF3,color='y',LineWidth=1.0,label='filtered')
# plt.show()
# plt.plot(df.loc[1500:3500, df.columns == 'AF3'])
plt.plot(aaa.loc[1500:1628, aaa.columns == 'AF3'], color='black',LineWidth=1.25)
plt.plot(dsp_data.loc[1500:1628, dsp_data.columns == 'FC5'], color='yellow',LineWidth=1.25)
# plt.plot(dsp_data.loc[1500:1628, dsp_data.columns == 'AF3'], color='orange',LineWidth=1.25)
# plt.plot(fft_data.loc[:, fft_data.columns == 'AF3'], color='orange',LineWidth=1.25)

In [ ]:
df_signals = df.loc[:, df.columns != 'Time128Hz'].to_numpy()
df_times = df.loc[:, df.columns == 'Time128Hz'].to_numpy()

# dsp_signals = dsp_data.loc[1500:3500, dsp_data.columns != 'Time128Hz'].to_numpy()
# dsp_times = dsp_data.loc[1500:3500, dsp_data.columns == 'Time128Hz'].to_numpy()
# spike1 = dsp_data.loc[1500:3500, dsp_data.columns == 'FC5']
dsp_signals = dsp_data.loc[:, dsp_data.columns != 'Time128Hz'].to_numpy()
dsp_times = dsp_data.loc[:, dsp_data.columns == 'Time128Hz'].to_numpy()

s = 3
offset = int(128 * 1)
# ts = offset + int(128 * s)

plt.figure()
for i in range(dsp_signals.shape[1]):
    plt.subplot(dsp_signals.shape[1], 1, i+1)
    plt.plot(dsp_times[offset:],dsp_signals[offset:, i],color='orange',LineWidth=1.5,label='Noisy')
    # plt.plot(df_times[offset:ts],df_signals[offset:ts, i],color='black',LineWidth=1.25,label='clean')
plt.show()

plt.plot(df['Time128Hz'], )



# for xy in range(0, 2, 1):
    #     plt.subplot(signals1.shape[1], 1, xy+1)
    #     plt.plot(t1[:128],signals1[:128, xy],color='orange',LineWidth=1.25,label='Noisy')
    #     plt.plot(t2[:128],signals2[:128, xy],color='black',LineWidth=1.0,label='clean')
    # plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Fixing random state for reproducibility
np.random.seed(19680801)

seconds = 2
nsamples = seconds * 128

dt = 0.01
t = t1[:nsamples]
# t = np.arange(0, 30, dt)
nse1 = np.random.randn(len(t))                 # white noise 1
nse2 = np.random.randn(len(t))                 # white noise 2

# Two signals with a coherent part at 10Hz and a random part
# s1 = np.sin(2 * np.pi * 10 * t) + nse1
# s2 = np.sin(2 * np.pi * 10 * t) + nse2
s1 = signals1[:nsamples,12]
s2 = signals2[:nsamples,2]
fig, axs = plt.subplots(2, 1)
axs[0].plot(t, s1,color='orange',LineWidth=1.50)#, t, s2,color='black',LineWidth=1.25)
axs[0].plot(t, s2,color='black',LineWidth=1.00)#, t, s2,color='black',LineWidth=1.25)
# axs[0].set_xlim(0, 7)
axs[0].set_xlabel('time')
axs[0].set_ylabel('s1 and s2')
axs[0].grid(True)

cxy, f = axs[1].cohere(s1, s2, 128, 1. / dt)
axs[1].set_ylabel('coherence')

fig.tight_layout()
plt.show()

In [ ]:
# # knn for predicting eye state
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import KFold
# from sklearn.neighbors import KNeighborsClassifier
# from numpy import mean
# # load the dataset
# data = pd.read_csv('/content/EEG_Eye_State_no_outliers.csv', header=None)
# values = data.values
# # evaluate knn using 10-fold cross-validation
# scores = list()
# kfold = KFold(10, shuffle=True, random_state=1)
# for train_ix, test_ix in kfold.split(values):
# 	# define train/test X/y
# 	trainX, trainy = values[train_ix, :-1], values[train_ix, -1]
# 	testX, testy = values[test_ix, :-1], values[test_ix, -1]
# 	# define model
# 	model = KNeighborsClassifier(n_neighbors=3)
# 	# fit model on train set
# 	model.fit(trainX, trainy)
# 	# forecast test set
# 	yhat = model.predict(testX)
# 	# evaluate predictions
# 	score = accuracy_score(testy, yhat)
# 	# store
# 	scores.append(score)
# 	print('>%.3f' % score)
# # calculate mean score across each run
# print('Final Score: %.3f' % (mean(scores)))

In [ ]:
# # knn for predicting eye state
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
# # load the dataset
# data = pd.read_csv('/content/EEG_Eye_State_no_outliers.csv', header=None)
# values = data.values
# # split data into inputs and outputs
# X, y = values[:, :-1], values[:, -1]
# # split the dataset
# trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=1)
# # define model
# model = KNeighborsClassifier(n_neighbors=3)
# # fit model on train set
# model.fit(trainX, trainy)
# # forecast test set
# yhat = model.predict(testX)
# # evaluate predictions
# score = accuracy_score(testy, yhat)
# print(score)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
# load the dataset
data = pd.read_csv('EEG_Eye_State_no_outliers.csv', header=None)
values = data.values
# split data into inputs and outputs
X, y = values[:, :-1], values[:, -1]
# split the dataset
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.1, shuffle=False, random_state=1)
# define model
model = KNeighborsClassifier(n_neighbors=3)
# fit model on train set
model.fit(trainX, trainy)
# forecast test set
yhat = model.predict(testX)
# evaluate predictions
score = accuracy_score(testy, yhat)
print(score)

# another input file

In [ ]:
raw_data = '/content/2_KA_VisualnaStimulaciq_14ch.csv'
all_cols = ['Time128Hz', 'Epoch', 'AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'EventId', 'EventDate', 'EventDuration']
def fix_eventid_col(data):
  # this should be a func
  data.columns = df.columns.str.replace(' ', '')
  EventID_NEW = ''
  EventID_TO_SET = ''
  for index, row in data.iterrows(): #итерираме ред по ред с вградения метод iterrows() от pandas
      EventID_NEW = str(row['EventId']) # достъпваме данни чрез име на колона
      if EventID_NEW == '0':
          data.at[index, 'EventId'] = str(EventID_TO_SET)
      else:
          EventID_NEW = str(row['EventId'])
          EventID_TO_SET = str(row['EventId'])
  df.fillna(value=0, inplace=True)
  return data


def label_data(data):
  for index, row in data.iterrows(): #итерираме ред по ред с вградения метод iterrows() от pandas
      EventID_value = str(row['EventId']) # достъпваме данни чрез име на колона
      if EventID_value == '33025':
          # data.at[index, 'EventId'] = str(EventID_TO_SET)
          data.at[index, 'EventId'] = 0
      elif EventID_value == '33026':
          data.at[index, 'EventId'] = 1
      else:
          data.at[index, 'EventId'] = -1
  return data

In [ ]:
# we use pandas to read the dataset and specify its feature names
df = pd.read_csv(raw_data, header=0)
df2 = fix_eventid_col(df)
df3 = label_data(df2)
values = df3[['AF3', 'F7', 'F3', 'FC5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'EventId']].values


# all_cols = ['Time128Hz', 'Epoch', 'AF3', 'F7', 'F3', 'F5', 'T7', 'P7', 'O1', 'O2', 'P8', 'T8', 'FC6', 'F4', 'F8', 'AF4', 'EventId', 'EventDate', 'EventDuration']

In [ ]:
plt.figure()
for i in range(values.shape[1]):
	plt.subplot(values.shape[1], 1, i+1)
	plt.plot(values[:, i])
plt.show()

# test


In [ ]:
!pip install neurodsp

In [ ]:
#@title
from scipy import signal
import neurodsp
from neurodsp.utils import create_times


b, a = signal.butter(8, 30, 'low', analog=True)
w, h = signal.freqs(b, a)
plt.semilogx(w, 20 * np.log10(abs(h)))
plt.title('Butterworth filter frequency response')
plt.xlabel('Frequency [radians / second]')
plt.ylabel('Amplitude [dB]')
plt.margins(0, 0.1)
plt.grid(which='both', axis='both')
plt.axvline(100, color='green') # cutoff frequency
plt.show()


s1 = df3['AF3'].to_numpy()
print(s1)
fs = 128
times = create_times(len(s1)/fs, fs)

t = np.linspace(0, 1, 1000, False)  # 1 second
sig = np.sin(2*np.pi*10*t) + np.sin(2*np.pi*20*t)
sig = s1
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
ax1.plot(times, sig)
ax1.set_title('10 Hz and 20 Hz sinusoids')
# ax1.axis([0, 1, -2, 2])

sos = signal.butter(20, 50, 'hp', fs=128, output='sos')
filtered = signal.sosfilt(sos, sig)
ax2.plot(times, filtered)
ax2.set_title('After 15 Hz high-pass filter')
# ax2.axis([0, 1, -20, 20])
ax2.set_xlabel('Time [seconds]')
plt.tight_layout()
plt.show()


# sig = df3.AF4.values
# fs = 128
# times = create_times(len(sig)/fs, fs)

# # t = np.linspace(0,1,1000,False)

# fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True)
# ax1.plot(times, sig)
# ax1.set_title('10 Hz and 20 Hz sinusoids')
ax1.axis([0, 1, -2, 2])
# sos = signal.butter(8, 30, 'hp', fs=128, output='sos')
# filtered = signal.sosfilt(sos, sig)
# ax2.plot(times, filtered)
# ax2.set_title('After 15 Hz high-pass filter')
ax2.axis([0, 1, -2, 2])
# ax2.set_xlabel('Time [seconds]')
# plt.tight_layout()
# plt.show()
# print(times)

# EEG Plotly

In [ ]:
from plotly import tools
from plotly.graph_objs import Layout, YAxis, Scatter, Annotation, Annotations, Data, Figure, Marker, Font

In [ ]:
n_channels = 14


# Plottting example EEGLAB FILE

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import find_peaks

openbci_datapath = '../record.txt'
sample_rate = 250

data = pd.read_csv(openbci_datapath, sep=",", header=6, index_col=False, names=['index', 'ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8', 'A5', 'A6', 'A7', 'Time', 'TimeStamp'], usecols=['ch1', 'ch2', 'ch3', 'ch4', 'ch5', 'ch6', 'ch7', 'ch8', 'A7', 'TimeStamp'])
print("OpenBCI data shape: ", data.shape)